# Mutli-Class Comparative Analysis

## Data Pre-processing

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split

In [ ]:
# Define dataframes for csvs
nonattack_csv = pd.read_csv('./multiclass_test/comp_analysis/non-attack_random_sample.csv')
dos_csv = pd.read_csv('./multiclass_test/comp_analysis/dos_output_OS.csv')
analysis_csv = pd.read_csv('./multiclass_test/comp_analysis/analysis_output_OS.csv')
backdoors_csv = pd.read_csv('./multiclass_test/comp_analysis/backdoors_output_OS.csv')
exploits_csv = pd.read_csv('./multiclass_test/comp_analysis/exploits_output_OS.csv')
generic_csv = pd.read_csv('./multiclass_test/comp_analysis/generic_output_OS.csv')
reconnaissance_csv = pd.read_csv('./multiclass_test/comp_analysis/recon_output_OS.csv')
shellcode_csv = pd.read_csv('./multiclass_test/comp_analysis/shellcode_output_OS.csv')
fuzzers_csv = pd.read_csv('./multiclass_test/comp_analysis/fuzzers_output_OS.csv')

# Create copies of dataframes
nonattack_copy = nonattack_csv.copy().head(16000)
dos_copy = dos_csv.copy()
analysis_copy = analysis_csv.copy()
backdoors_copy = backdoors_csv.copy()
exploits_copy = exploits_csv.copy()
generic_copy = generic_csv.copy()
reconnaissance_copy = reconnaissance_csv.copy()
shellcode_copy = shellcode_csv.copy()
fuzzers_copy = fuzzers_csv.copy()

# Correct labels of dataframes
analysis_copy['Label'] = 0
backdoors_copy['Label'] = 1
dos_copy['Label'] = 2
exploits_copy['Label'] = 3
fuzzers_copy['Label'] = 4
generic_copy['Label'] = 5
nonattack_copy['Label'] = 6
reconnaissance_copy['Label'] = 7
shellcode_copy['Label'] = 8

In [ ]:
# Drop last two columns (attack category and data label)
nonattack_to_normal = nonattack_copy.drop(columns=['attack_cat', 'Label'])
dos_to_normal = dos_copy.drop(columns=['attack_cat', 'Label'])
analysis_to_normal = analysis_copy.drop(columns=['attack_cat', 'Label'])
backdoors_to_normal = backdoors_copy.drop(columns=['attack_cat', 'Label'])
exploits_output_OS = exploits_copy.drop(columns=['attack_cat', 'Label'])
generic_to_normal = generic_copy.drop(columns=['attack_cat', 'Label'])
reconnaissance_to_normal = reconnaissance_copy.drop(columns=['attack_cat', 'Label'])
shellcode_to_normal = shellcode_copy.drop(columns=['attack_cat', 'Label'])
fuzzers_to_normal = fuzzers_copy.drop(columns=['attack_cat', 'Label'])

# Normalise data
nonattack_normal = nonattack_to_normal / 255
dos_normal = dos_to_normal / 255
analysis_normal = analysis_to_normal / 255
backdoors_normal = backdoors_to_normal / 255
exploits_normal = exploits_output_OS / 255
generic_normal = generic_to_normal / 255
reconnaissance_normal = reconnaissance_to_normal / 255
shellcode_normal = shellcode_to_normal / 255
fuzzers_normal = fuzzers_to_normal / 255

# Add dropped columns back
nonattack_normal = pd.concat([nonattack_normal, nonattack_copy[['attack_cat', 'Label']]], axis=1)
dos_normal = pd.concat([dos_normal, dos_copy[['attack_cat', 'Label']]], axis=1)
analysis_normal = pd.concat([analysis_normal, analysis_copy[['attack_cat', 'Label']]], axis=1)
backdoors_normal = pd.concat([backdoors_normal, backdoors_copy[['attack_cat', 'Label']]], axis=1)
exploits_normal = pd.concat([exploits_normal, exploits_copy[['attack_cat', 'Label']]], axis=1)
generic_normal = pd.concat([generic_normal, generic_copy[['attack_cat', 'Label']]], axis=1)
reconnaissance_normal = pd.concat([reconnaissance_normal, reconnaissance_copy[['attack_cat', 'Label']]], axis=1).head(13940)
shellcode_normal = pd.concat([shellcode_normal, shellcode_copy[['attack_cat', 'Label']]], axis=1)
fuzzers_normal = pd.concat([fuzzers_normal, fuzzers_copy[['attack_cat', 'Label']]], axis=1)

nonattack_normal.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,0.820513,0.476463,0.511628,0.732197,0.894737,0.076923,0.000017,0.000010,0.000011,0.121569,...,0.0,0.030303,0.075758,0.030303,0.060606,0.0,0.0,0.000000,NaN,6
1,0.974359,0.160601,0.558140,0.255702,0.849624,0.307692,0.025600,0.002870,0.000224,0.121569,...,0.0,0.000000,0.045455,0.045455,0.045455,0.0,0.0,0.015152,NaN,6
2,0.769231,0.027695,0.465116,0.018470,0.894737,0.076923,0.000075,0.000044,0.000021,0.121569,...,0.0,0.045455,0.045455,0.015152,0.015152,0.0,0.0,0.015152,NaN,6
3,0.820513,0.315975,0.581395,0.205011,0.850746,0.333333,0.000022,0.000123,0.000200,0.121569,...,0.0,0.046512,0.146341,0.000000,0.000000,0.0,0.0,0.000000,NaN,6
4,0.974359,0.457786,0.181818,0.008689,0.849624,0.357143,0.001126,0.000022,0.000127,0.121569,...,0.0,0.032258,0.032787,0.016949,0.084746,0.0,0.0,0.048387,NaN,6


In [ ]:
# Train test split 70:30
nonattack_train_split = nonattack_normal.head(11200)
dos_train_split = dos_normal.head(11200)
analysis_train_split = analysis_normal.head(11200)
backdoors_train_split = backdoors_normal.head(11200)
exploits_train_split = exploits_normal.head(11200)
generic_train_split = generic_normal.head(11200)
reconnaissance_train_split = reconnaissance_normal.head(9760)
shellcode_train_split = shellcode_normal.head(11200)
fuzzers_train_split = fuzzers_normal.head(11200)

nonattack_test_split = nonattack_normal.tail(4800)
dos_test_split = dos_normal.tail(4800)
analysis_test_split = analysis_normal.tail(4800)
backdoors_test_split = backdoors_normal.tail(4800)
exploits_test_split = exploits_normal.tail(4800)
generic_test_split = generic_normal.tail(4800)
reconnaissance_test_split = reconnaissance_normal.tail(4180)
shellcode_test_split = shellcode_normal.tail(4800)
fuzzers_test_split = fuzzers_normal.tail(4800)

In [ ]:
# Train set - validatin split 80:20, create X and y sets
X_train = pd.concat([nonattack_train_split.head(8960), dos_train_split.head(8960)], axis=0)
X_train = pd.concat([X_train, analysis_train_split.head(8960)], axis=0)
X_train = pd.concat([X_train, backdoors_train_split.head(8960)], axis=0)
X_train = pd.concat([X_train, exploits_train_split.head(8960)], axis=0)
X_train = pd.concat([X_train, generic_train_split.head(8960)], axis=0)
X_train = pd.concat([X_train, reconnaissance_train_split.head(7820)], axis=0)
X_train = pd.concat([X_train, shellcode_train_split.head(8960)], axis=0)
X_train = pd.concat([X_train, fuzzers_train_split.head(8960)], axis=0)

y_train = X_train[['Label']]
X_train = X_train.drop(columns=['attack_cat', 'Label'])

In [ ]:
# Validation set - validation split 80:20, create X and y sets
X_val = pd.concat([nonattack_train_split.tail(2240), dos_train_split.tail(2240)], axis=0)
X_val = pd.concat([X_val, analysis_train_split.tail(2240)], axis=0)
X_val = pd.concat([X_val, backdoors_train_split.tail(2240)], axis=0)
X_val = pd.concat([X_val, exploits_train_split.tail(2240)], axis=0)
X_val = pd.concat([X_val, generic_train_split.tail(2240)], axis=0)
X_val = pd.concat([X_val, reconnaissance_train_split.tail(1940)], axis=0)
X_val = pd.concat([X_val, shellcode_train_split.tail(2240)], axis=0)
X_val = pd.concat([X_val, fuzzers_train_split.tail(2240)], axis=0)

y_val = X_val[['Label']]
X_val = X_val.drop(columns=['attack_cat', 'Label'])

In [ ]:
# Reshape data to 20 row vectors for fair comparison, one-hot encode y labels
def reshape_data(X, y):
    # Reshape X
    X_array = X.to_numpy()
    samples, features = X.shape
    X_reshaped = X_array.reshape(-1, 20, features)

    # Reshape y
    y_array = y.to_numpy().flatten()
    y_reshaped = y_array[::20]

    # One-hot encode y
    y_one_hot = tf.keras.utils.to_categorical(y_reshaped, num_classes=9)

    return X_reshaped, y_one_hot

# Reshape and encode training data
X_train_reshaped, y_train_encoded = reshape_data(X_train, y_train['Label'])

# Reshape and encode validation data
X_val_reshaped, y_val_encoded = reshape_data(X_val, y_val['Label'])

In [ ]:
# Test set
X_test = pd.concat([nonattack_test_split, dos_test_split], axis=0)
X_test = pd.concat([X_test, analysis_test_split], axis=0)
X_test = pd.concat([X_test, backdoors_test_split], axis=0)
X_test = pd.concat([X_test, exploits_test_split], axis=0)
X_test = pd.concat([X_test, generic_test_split], axis=0)
X_test = pd.concat([X_test, reconnaissance_test_split], axis=0)
X_test = pd.concat([X_test, shellcode_test_split], axis=0)
X_test = pd.concat([X_test, fuzzers_test_split], axis=0)

y_test = X_test[['Label']]
X_test = X_test.drop(columns=['attack_cat', 'Label'])

# Reshape and encode test data
X_test_reshaped, y_test_encoded = reshape_data(X_test, y_test['Label'])

In [ ]:
# Early stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

## Artificial Neural Network (ANN)

In [ ]:
# Model Architecture (from Sydney et al.)
model = Sequential([
    layers.Flatten(input_shape=(20, 47)),
    Dense(150, activation='relu'),
    Dense(9, activation='softmax')
])

# Create and compile the best performing model (with 150 neurons)
optimizer = Adam(learning_rate=0.02)

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Save results to CSV
csv_filename = './multiclass_test/comp_analysis/comp_ann_results.csv'


# Define the column names
fieldnames = ['trial_num', 'precision', 'recall', 'Analysis', 'Backdoors', 'DoS', 
              'Exploits', 'Fuzzers', 'Generic', 'Non-attack', 'Reconnaissance', 'Shellcode']

column_map = {
                0: 'Analysis',
                1: 'Backdoors',
                2: 'DoS',
                3: 'Exploits',
                4: 'Fuzzers',
                5: 'Generic',
                6: 'Non-attack',
                7: 'Reconnaissance',
                8: 'Shellcode'
            }


with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for n in range(30):
        # Compile the model
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        ANN_history = model.fit(X_train_reshaped, 
                                y_train_encoded, 
                                epochs=50, 
                                batch_size=32, 
                                validation_data=(X_val_reshaped, y_val_encoded), 
                                callbacks=[early_stopping])
        
        # Make Predictions
        ANN_predictions = model.predict(X_test_reshaped)
        ANN_binary_predictions = np.where(ANN_predictions > 0.5, 1, 0)
        integer_predictions = np.argmax(ANN_binary_predictions, axis=1)

        # Flatten y_test
        true_labels = np.argmax(y_test_encoded, axis=1).squeeze()
        
        # Overall precision and recall
        precision = precision_score(true_labels, integer_predictions, average='macro')
        recall = recall_score(true_labels, integer_predictions, average='macro')
        
        # Calculate confusion matrix
        conf_matrix = confusion_matrix(true_labels, integer_predictions)

        # Prepare the row data
        row_data = {
            'trial_num': n,
            'precision': precision,
            'recall': recall
        }

        # Metrics for each class
        for i in range(conf_matrix.shape[0]):
            TP = conf_matrix[i, i]
            FP = conf_matrix[:, i].sum() - TP
            FN = conf_matrix[i, :].sum() - TP
            TN = conf_matrix.sum() - (FP + FN + TP)
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0
        
            # Calculate precision and recall
            class_precision = TP / (TP + FP) if (TP + FP) != 0 else 0
            class_recall = TP / (TP + FN) if (TP + FN) != 0 else 0

            # Add class-specific metrics to the row data
            row_data[column_map[i]] = f'TP: {TP} - FP: {FP} - FN: {FN} - TN: {TN} - FPR: {FPR:.4f} - Pr: {class_precision:.4f} - Re: {class_recall:.4f}'


        # Write the row to the CSV file
        writer.writerow(row_data)
        
        print(f"Trial {n} saved to {csv_filename}")

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4957 - loss: 1.9407 - val_accuracy: 0.8308 - val_loss: 0.5956
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7934 - loss: 0.5973 - val_accuracy: 0.8590 - val_loss: 0.5331
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8216 - loss: 0.4887 - val_accuracy: 0.8278 - val_loss: 0.6200
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8530 - loss: 0.4108 - val_accuracy: 0.8671 - val_loss: 0.5000
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8364 - loss: 0.4748 - val_accuracy: 0.8973 - val_loss: 0.4504
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8470 - loss: 0.3845 - val_accuracy: 0.8892 - val_loss: 0.5335
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8699 - loss: 0.3459 - val_accuracy: 0.8610 - val_loss: 0.6347
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8643 - loss: 0.3842 - val_accuracy: 0.

In [ ]:
## Long Short Term Memory

## Long Short-Term Memory (LSTM)

In [ ]:
# Model architecture (from Jallad et al.)
model = Sequential([
    LSTM(64, activation='relu', input_shape=(20, 47)),
    Dropout(0.5),
    Dense(9, activation='softmax')
])

# Compile
optimizer = RMSprop(learning_rate=0.001, rho=0.9, decay=0.0)

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
# Save results to CSV
csv_filename = './multiclass_test/comp_analysis/comp_lstm_results.csv'


# Define the column names
fieldnames = ['trial_num', 'precision', 'recall', 'Analysis', 'Backdoors', 'DoS', 
              'Exploits', 'Fuzzers', 'Generic', 'Non-attack', 'Reconnaissance', 'Shellcode']

column_map = {
                0: 'Analysis',
                1: 'Backdoors',
                2: 'DoS',
                3: 'Exploits',
                4: 'Fuzzers',
                5: 'Generic',
                6: 'Non-attack',
                7: 'Reconnaissance',
                8: 'Shellcode'
            }


with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for n in range(30):
        # Compile the model
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        LSTM_history = model.fit(X_train_reshaped, 
                                y_train_encoded, 
                                epochs=50, 
                                batch_size=32, 
                                validation_data=(X_val_reshaped, y_val_encoded), 
                                callbacks=[early_stopping])
        
        # Make Predictions
        LSTM_predictions = model.predict(X_test_reshaped)
        LSTM_binary_predictions = np.where(LSTM_predictions > 0.5, 1, 0)
        integer_predictions = np.argmax(LSTM_binary_predictions, axis=1)

        # Flatten y_test
        true_labels = np.argmax(y_test_encoded, axis=1).squeeze()
        
        # Overall precision and recall
        precision = precision_score(true_labels, integer_predictions, average='macro')
        recall = recall_score(true_labels, integer_predictions, average='macro')
        
        # Calculate confusion matrix
        conf_matrix = confusion_matrix(true_labels, integer_predictions)

        # Prepare the row data
        row_data = {
            'trial_num': n,
            'precision': precision,
            'recall': recall
        }

        # Metrics for each class
        for i in range(conf_matrix.shape[0]):
            TP = conf_matrix[i, i]
            FP = conf_matrix[:, i].sum() - TP
            FN = conf_matrix[i, :].sum() - TP
            TN = conf_matrix.sum() - (FP + FN + TP)
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0
        
            # Calculate precision and recall
            class_precision = TP / (TP + FP) if (TP + FP) != 0 else 0
            class_recall = TP / (TP + FN) if (TP + FN) != 0 else 0

            # Add class-specific metrics to the row data
            row_data[column_map[i]] = f'TP: {TP} - FP: {FP} - FN: {FN} - TN: {TN} - FPR: {FPR:.4f} - Pr: {class_precision:.4f} - Re: {class_recall:.4f}'


        # Write the row to the CSV file
        writer.writerow(row_data)
        
        print(f"Trial {n} saved to {csv_filename}")

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2971 - loss: 1.8559 - val_accuracy: 0.4914 - val_loss: 1.3542
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5457 - loss: 1.1878 - val_accuracy: 0.6737 - val_loss: 0.9280
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6568 - loss: 0.9095 - val_accuracy: 0.7815 - val_loss: 0.6564
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Trial 0 saved to ./multiclass_test/comp_analysis/comp_lstm_results.csv
Epoch 1/50


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5441 - loss: 1.1925 - val_accuracy: 0.6939 - val_loss: 1.0845
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6658 - loss: 0.8958 - val_accuracy: 0.6818 - val_loss: 1.3883
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7260 - loss: 0.8477 - val_accuracy: 0.6918 - val_loss: 1.2718
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Trial 1 saved to ./multiclass_test/comp_analysis/comp_lstm_results.csv
Epoch 1/50


c:\Users\Admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6543 - loss: 0.9336 - val_accuracy: 0.6999 - val_loss: 1.6575
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7315 - loss: 0.8269 - val_accuracy: 0.8026 - val_loss: 0.6312
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7838 - loss: 0.6520 - val_accuracy: 0.8097 - val_loss: 0.5882
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Trial 2 saved to ./multiclass_test/comp_analysis/comp_lstm_results.csv
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7103 - loss: 0.8316 - val_accuracy: 0.7956 - val_loss: 0.6441
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7733 - loss: 0.6609 - val_accuracy: 0.7986 - val_loss: 0.6402
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7877 - loss: 0.6507 - val_accuracy: 0.8359 - val_loss: 0.8947
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Trial 3 saved to ./multiclass_test/comp_analysis/comp_lstm_results.csv
Epoch 1/50
125/125 ━━━━

In [ ]:
## 1D CNN

## 1D Convolutional Neural Network (CNN)

In [ ]:
# Model architecture
model = models.Sequential()
model.add(layers.Conv1D(64, 1, activation='relu', input_shape=(20, 47)))
model.add(layers.MaxPooling1D(1))

model.add(layers.Conv1D(64, 1, activation='tanh'))
model.add(layers.MaxPooling1D(1))

model.add(layers.Conv1D(96, 1, activation='tanh'))
model.add(layers.MaxPooling1D(1))

model.add(layers.Dropout(0.3))

# Flatten Layer
model.add(layers.Flatten())

# Dense Layer
model.add(layers.Dense(96, activation='relu'))
model.add(layers.Dropout(0.3))

# Output Layer
model.add(layers.Dense(9, activation='softmax'))

# Choose optimizer and learning rate
learning_rate = 0.0011228308916583806
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Save results to CSV
csv_filename = './multiclass_test/comp_analysis/comp_1DCNN_results.csv'


# Define the column names
fieldnames = ['trial_num', 'precision', 'recall', 'Analysis', 'Backdoors', 'DoS', 
              'Exploits', 'Fuzzers', 'Generic', 'Non-attack', 'Reconnaissance', 'Shellcode']

column_map = {
                0: 'Analysis',
                1: 'Backdoors',
                2: 'DoS',
                3: 'Exploits',
                4: 'Fuzzers',
                5: 'Generic',
                6: 'Non-attack',
                7: 'Reconnaissance',
                8: 'Shellcode'
            }


with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for n in range(30):
        # Compile the model
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        CNN_history = model.fit(X_train_reshaped, 
                                y_train_encoded, 
                                epochs=50, 
                                batch_size=32, 
                                validation_data=(X_val_reshaped, y_val_encoded), 
                                callbacks=[early_stopping])

        
        # Make Predictions
        CNN_predictions = model.predict(X_test_reshaped)
        CNN_binary_predictions = np.where(CNN_predictions > 0.5, 1, 0)
        integer_predictions = np.argmax(CNN_binary_predictions, axis=1)

        # Flatten y_test
        true_labels = np.argmax(y_test_encoded, axis=1).squeeze()
        
        # Overall precision and recall
        precision = precision_score(true_labels, integer_predictions, average='macro')
        recall = recall_score(true_labels, integer_predictions, average='macro')
        
        # Calculate confusion matrix
        conf_matrix = confusion_matrix(true_labels, integer_predictions)

        # Prepare the row data
        row_data = {
            'trial_num': n,
            'precision': precision,
            'recall': recall
        }

        # Metrics for each class
        for i in range(conf_matrix.shape[0]):
            TP = conf_matrix[i, i]
            FP = conf_matrix[:, i].sum() - TP
            FN = conf_matrix[i, :].sum() - TP
            TN = conf_matrix.sum() - (FP + FN + TP)
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0
        
            # Calculate precision and recall
            class_precision = TP / (TP + FP) if (TP + FP) != 0 else 0
            class_recall = TP / (TP + FN) if (TP + FN) != 0 else 0

            # Add class-specific metrics to the row data
            row_data[column_map[i]] = f'TP: {TP} - FP: {FP} - FN: {FN} - TN: {TN} - FPR: {FPR:.4f} - Pr: {class_precision:.4f} - Re: {class_recall:.4f}'


        # Write the row to the CSV file
        writer.writerow(row_data)
        
        print(f"Trial {n} saved to {csv_filename}")

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4784 - loss: 1.4649 - val_accuracy: 0.7311 - val_loss: 0.7050
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7602 - loss: 0.6610 - val_accuracy: 0.7523 - val_loss: 0.6217
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8164 - loss: 0.5140 - val_accuracy: 0.8036 - val_loss: 0.5520
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Trial 0 saved to ./multiclass_test/comp_analysis/comp_1DCNN_results.csv
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7730 - loss: 0.6484 - val_accuracy: 0.7976 - val_loss: 0.4913
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8043 - loss: 0.5114 - val_accuracy: 0.9225 - val_loss: 0.3733
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8309 - loss: 0.4385 - val_accuracy: 0.8671 - val_loss: 0.4767
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8596 - loss: 0.3859 - val_accuracy: 0.8842 - val_loss: 